In [1]:
# install qdrant
!pip install qdrant-client -qU

In [2]:
import platform
platform.version(), platform.release()

('Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:23 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T6020',
 '22.5.0')

In [3]:
!python --version

Python 3.8.16


In [4]:
# initialize client

from qdrant_client import models, QdrantClient

client = QdrantClient(":memory:")

In [5]:
# create a collection

from qdrant_client.http.models import Distance, VectorParams

client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [6]:
# add vectors
from qdrant_client.http.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={"city": ["Berlin", "London"]},
        ),
        PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={"city": ["Berlin", "Moscow"]},
        ),
        PointStruct(
            id=4,
            vector=[0.18, 0.01, 0.85, 0.80],
            payload={"city": ["London", "Moscow"]},
        ),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"count": [0]}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44]),
    ],
)

In [7]:
# run a query
search_result = client.search(
    collection_name="test_collection", query_vector=[0.2, 0.1, 0.9, 0.7], limit=3
)

In [8]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue


search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    limit=3,
)

In [9]:
must_conditions = [FieldCondition(key="city", match=MatchValue(value="London"))]
print(type(must_conditions[0]))

<class 'qdrant_client.http.models.models.FieldCondition'>
